In [ ]:
!pip install -e ../../


In [ ]:
!wget https://raw.githubusercontent.com/akihironitta/xView-PyTorch/refs/heads/master/datasets.py
!mv datasets.py ../../terratorch/datasets/xview.py


In [ ]:
from terratorch.datamodules.xview import XviewDataModule
dm = XviewDataModule()